In [ ]:
import futu as ft
import pandas as pd
import os

In [ ]:
# 实例化行情上下文对象
# 如下范例，创建api行情对象，调用get_market_snapshot获取港股腾讯00700的报价快照数据,最后关闭对象
quote_ctx = ft.OpenQuoteContext(host="127.0.0.1", port=11111)
print(quote_ctx.get_market_snapshot('HK.00700'))
quote_ctx.close()

In [ ]:
#  Get list of all hong kong stock
"""
all_stock_hk_plate = 'HK.BK1910'
quote_ctx = ft.OpenQuoteContext(host="127.0.0.1", port=11111)
stock_list_all_col = quote_ctx.get_plate_stock(all_stock_hk_plate)[1]

# stock_type
#  STOCK：  股票
#  IDX： 指数
#  ETF： 交易所交易基金(Exchange Traded Funds)
#  WARRANT： 港股涡轮牛熊界内证
#  BOND： 债券
#  DRVT： 期权
#  NONE： 未知
stock_list_all_col = stock_list_all_col[stock_list_all_col['stock_type'] == 'STOCK']
stock_list = stock_list_all_col[['code', 'lot_size', 'stock_name', 'stock_type', 'stock_id']]
print(stock_list)
quote_ctx.close()
"""

In [ ]:
# stock_list.to_csv('stock_hk_list.csv')

In [ ]:
import time
# change rate
def get_all_change_list(quote_ctx, codes, start='2019-10-04', end='2019-10-04', max_count=10, page_req_key=None, stock_num=100):
    change_rates = []
    
    stock_cnt = 0
    for code in codes:
        if stock_cnt > stock_num:
            break
        print('{}/{}'.format(stock_cnt, stock_num))
        history_kline = quote_ctx.request_history_kline(code, start=start, end=end, max_count=1000, page_req_key=None)
        # Skip if  ret != RET_OK or data is empty
        if history_kline[0] == -1 or history_kline[1].empty:
            print(history_kline[1])
            continue
        history_kline = history_kline[1][['code', 'change_rate']].values.tolist()[0]
        change_rates.append(history_kline)
        stock_cnt += 1
        time.sleep(3)
    
    change_rates = pd.DataFrame(change_rates, columns=['code', 'change_rate'])
    return change_rates.sort_values(by=['change_rate'], ascending=False)

stock_list = pd.read_csv('stock_hk_list.csv')

quote_ctx = ft.OpenQuoteContext(host="127.0.0.1", port=11111)
change_rates_sorted = get_all_change_list(quote_ctx, stock_list['code'])
print(change_rates_sorted)

quote_ctx.close()

In [ ]:
# Draw trend of change rate for one stock
!python3 -mpip install matplotlib
import matplotlib.pyplot as plt

def draw_history_change_rate(quote_ctx, code, start='2019-09-04', end='2019-10-04', max_count=1000):
    history_kline = quote_ctx.request_history_kline(code, start=start, end=end, max_count=max_count)
    if history_kline[0] == -1 or history_kline[1].empty:
        print('No data')
        return
    history_kline_data = history_kline[1][['change_rate', 'time_key']]
    
    history_kline_data.plot(x='time_key', y='change_rate')
    
    #df = history_kline_data.cumsum()
    #plt.figure()
    #df.plot()

quote_ctx = ft.OpenQuoteContext(host="127.0.0.1", port=11111)
draw_history_change_rate(quote_ctx, 'HK.00022')
quote_ctx.close()

In [ ]:
# track stock price in real time

columns_name = ['code', 'time', 'is_bank', 'opened_mins', 'cur_price', 'last_close', 'avg_price', 'volume', 'turnover']
code_list = ['HK.00020', 'HK.00021', 'HK.00022']

def track_stock(quote_ctx, code, max_rows=100, max_fail=10):
    
    history_data = pd.DataFrame([], columns=columns_name)
    
    fail_cnt = 0
    while True:
        rt_data = quote_ctx.get_rt_data(code)
        if fail_cnt > max_fail:
            print('No data can be accessable')
            break
        if rt_data[0] < 1 or rt_data[1].empty:
            print('getting data failed')
            fail_cnt += 1
            continue
        history_data = pd.concat([history_data, rt_data[1]])
        
        if history_data.shape[0] > max_rows:
            history_data = pd.DataFrame([], columns=columns_name)

quote_ctx = ft.OpenQuoteContext(host="127.0.0.1", port=11111)
for code in code_list:
    print('{}'.format(code))
    track_stock(quote_ctx, code)

In [ ]:
# Train a predi